<a href="https://colab.research.google.com/github/sheikmohdimran/Experiments_2021/blob/main/NLP/Wav2Vec_ONNX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install -q transformers datasets jiwer onnxruntime soundfile

     |████████████████████████████████| 4.5 MB 9.2 MB/s 


In [ ]:
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import soundfile as sf
import torch
from jiwer import wer

In [ ]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
model.to("cuda");

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:419: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
librispeech_eval  = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")

def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch

librispeech_eval = librispeech_eval.map(map_to_array)

Reusing dataset librispeech_asr (/root/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/468ec03677f46a8714ac6b5b64dba02d246a228d92cbbad7f3dc190fa039eab1)
Loading cached processed dataset at /root/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/468ec03677f46a8714ac6b5b64dba02d246a228d92cbbad7f3dc190fa039eab1/cache-83ab6027297f59f2.arrow


In [ ]:
def map_to_pred(batch):
    input_values = tokenizer(batch["speech"], return_tensors="pt", padding="longest").input_values
    with torch.no_grad():
        logits = model(input_values.to("cuda")).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)
    batch["transcription"] = transcription
    return batch

In [ ]:
result = librispeech_eval.map(map_to_pred, batched=True, batch_size=1, remove_columns=["speech"])

print("WER:", wer(result["text"], result["transcription"]))


  0%|          | 0/73 [00:00<?, ?ba/s]

WER: 0.05304347826086957


In [ ]:
result["transcription"][0]

'A MAN SAID TO THE UNIVERSE SIR I EXIST'

In [ ]:
input_values=tokenizer(librispeech_eval["speech"][0], return_tensors="pt", padding="longest").input_values

In [ ]:
input_values=torch.zeros([1,470400])

In [ ]:
torch.onnx.export(model.to("cpu"), input_values.to("cpu"),  'wav2vec2-base.onnx')

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:443: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_weights.size() != (bsz * self.num_heads, tgt_len, src_len):
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:480: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_output.size() != (bsz * self.num_heads, tgt_len, self.head_dim):


In [ ]:
import onnxruntime
import numpy as np

In [ ]:
onnx_model="./wav2vec2-base.onnx"
session = onnxruntime.InferenceSession(onnx_model, None)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

In [ ]:
input_values=tokenizer(librispeech_eval["speech"][0], return_tensors="pt", padding="longest").input_values

In [ ]:
470400-input_values.numpy().shape[1]

396000

In [ ]:
#len(max(librispeech_eval['speech'],key=len))
import numpy as np
np.pad(input_values.numpy(), (396000), 'constant').shape


In [ ]:
logits_onnx = session.run([output_name], {input_name: np.pad(input_values.numpy(), (0, 396000), 'constant')})

In [ ]:
predicted_ids_onnx = np.argmax(logits_onnx, axis=-1)
transcription = tokenizer.batch_decode(predicted_ids_onnx[0])
transcription[0]

'A MAN SAID TO THE UNIVERSE SIR I EXIST A MAN SAID TO THE UNIVERSE SIR I EXIST A MAN SAID TO THE UNIVERSE SIR I EXIST A MAN SAID TO THE UNIVERSE SIR I EXIST A MAN SAID TO THE UNIVERSE SIR I EXIST A MAN SAID TO THE UNIVERSE SIR I EXIST A MAN SAID TO THE'

In [ ]:
def map_to_pred_onnx(batch):
    input_values = tokenizer(batch["speech"], return_tensors="pt", padding="longest").input_values
    input_values=np.resize(input_values.numpy(),(1,470400))
    print(input_values.shape)
    logits_onnx = session.run([output_name], {input_name: input_values})
    predicted_ids = np.argmax(logits_onnx, axis=-1)
    transcription = tokenizer.batch_decode(predicted_ids[0])
    batch["transcription"] = transcription
    return batch

In [ ]:
result = librispeech_eval.map(map_to_pred_onnx, batched=True, batch_size=1, remove_columns=["speech"])

print("WER:", wer(result["text"], result["transcription"]))


  0%|          | 0/73 [00:00<?, ?ba/s]

(1, 470400)


InvalidArgument: ignored